# 生成式深度学习

## 使用LSTM生成文本

### 采样策略的重要性

In [1]:
# 对于不同的softmax温度，对概率进行重新加权
import numpy as np
def reweight_distribution(original_distribution,temperature=0.5):
    distribution = np.log(original_distribution)/temperature
    distribution = np.exp(distribution)
    return distribution/np.sum(distributionribution)

### 实现字符级的LSTM文本生成

#### 准备数据

In [2]:
import keras
path = keras.utils.get_file('nietzsche.txt',origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:',len(text))

Using TensorFlow backend.


Corpus length: 600893


In [3]:
# 将字符序列向量化
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])   
print('Number of sequences:',len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:',len(chars))
char_indices = dict((char,chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1

Number of sequences: 200278
Unique characters: 58
Vectorization...


#### 构建网络

In [4]:
# 用于预测下一个字符的单层LSTM模型
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='relu'))
optimizer = keras.optimizers.RMSprop(lr=1e-2)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


#### 训练语言模型并从中采样

In [5]:
# 给定模型预测，采样下一个字符的函数
def sample(preds,temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [ ]:
# 文本生成循环
import random
import sys 


model.fit(x,y,batch_size=128,epochs=1)
start_index = random.randint(0,len(text)-maxlen-1)
generated_text = text[start_index:start_index+maxlen]
print('--- Generate with seed:"'+generated_text+'"')

for temperature in [0.2,0.5,1.0,1.2]:
    print('------ temperature:',temperature)
    sys.stdout.write(generated_text)

    for i in range(400):
        sampled = np.zeros((1,maxlen,len(chars)))
        for t,char in enumerate(generated_text):
            sampled[0,t,char_indices[char]]=1

        preds = model.predict(sampled,verbose=0)[0]
        next_index = sample(preds,temperature)
        next_char = chars[next_index]

        generated_text+=next_char
        generated_text=generated_text[1:]
        sys.stdout.write(next_char)

## DeepDream

### 用Keras实现DeepDream

In [6]:
# 加载预训练的Inception V3 模型
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',include_top=False)


In [7]:
# 设置DeepDream配置
layer_contributions ={
    'mixed2':0.2,
    'mixed3':3.,
    'mixed4':2.,
    'mixed5':1.5,
}

In [8]:
# 定义需要最大化的损失
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)

for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation),'float32'))
    loss += coeff*K.sum(K.square(activation[:,2:-2,2:-2,:]))/scaling

In [9]:
# 梯度上升过程
dream = model.input

grads = K.gradients(loss,dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)),1e-7)

outputs = [loss,grads]
fetch_loss_and_grads = K.function([dream],outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value,grad_values

def gradient_ascent(x,iterations,step,max_loss = None):
    for i in range(iterations):
        loss_value,grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value>max_loss:
            break
        print('...Loss value at ',i,':',loss_value)
        x += step*grad_values
    return x

In [10]:
# 辅助函数
import scipy
from keras.preprocessing import image
def resize_img(img,size):
    img =np.copy(img)
    factors = (1,float(size[0])/img.shape[1],float(size[1])/img.shape[2],1)
    return scipy.ndimage.zoom(img,factors,order=1)

def save_img(img,fname):
    pil_name = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname,pil_name)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x=x.reshape((3,x.shape[2],x.shape[3]))
        x=x.transpose((1,2,0))
    else:
        x = x.reshape((x.shape[1],x.shape[2],3))
    x /=2.
    x+=0.5
    x*=255.
    x=np.clip(x,0,255).astype('uint8')
    return x

In [ ]:
# 在多个连续尺度上运行梯度上升
import numpy as np
step = 0.01
num_octave = 3
octave_scale =1.4
iterations = 20

max_loss = 10.

base_image_path = './data/deepdream/'

img = preprocess_image(base_image_path)

original_shape=img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1,num_octave):
    shape = tuple([int(dim/(octave_scale**i)) for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img,successive_shapes[0])
for shape in successive_shapes:
    print('Processing image shape',shape)
    img = resize_img(img,shape)
    img = gradient_ascent(img,iterations=iterations,step=step,max_loss = max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img,shape)
    same_size_original = resize_img(original_img,shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img,shape)
    save_img(img ,fname='dream_at_scale_'+str(shape)+'.png')
    
save_img(img,fname='final_dream.png')